In [ ]:
using Newtonsoft.Json;

// Your JSON string
string jsonString = "{ 'company_id': '1470', 'bank_id': '1', 'amount': '100', 'company_order_num': '1', 'company_user': '1', 'estimated_payment_bank': '1', 'deposit_mode': '2', 'group_id': '0', 'web_url': 'https://pay.n8stg.com/Common/NotifyWithdrawal/Pay', 'memo': '1', 'key': '9586db5566ef115b824528b45ad67f60', 'terminal': '1'}";



// Deserialize the JSON string into an object
JuBaoPenPayDepositSubmissionRequest jsonstr = JsonConvert.DeserializeObject<JuBaoPenPayDepositSubmissionRequest>(jsonString);

// Now you can access the properties of the object
Console.WriteLine($"Name: {jsonstr.company_id}");



## Sample Request Payment Gateway

***JuBaoPen***
```C#
"{ 
'company_id': '1470', 
'bank_id': '1', 
'amount': '100', 
'company_order_num': '1', 
'company_user': '1', 
'estimated_payment_bank': '1', 
'deposit_mode': '2', 
'group_id': '0', 
'web_url': 'https://pay.n8stg.com/Common/NotifyWithdrawal/Pay', 
'memo': '1', 
'key': '9586db5566ef115b824528b45ad67f60', 
'terminal': '1'
}"

"{ 
'company_id': 'Bar', 
'bank_id': 'Bar2', 
'amount': 'Bar2', 
'company_order_num': 'Bar2', 
'company_user': 'Bar2', 
'card_num': 'Bar2', 
'card_name': 'Bar2', 
'web_url': 'Bar2', 
'Fooo': 'Bar2', 
'Fooo': 'Bar2', 
'Fooo': 'Bar2'
}"

```

In [ ]:
// JuBaoPen Models

public class JuBaoPenPayDepositSubmissionRequest
{
    internal string company_id { get; set; }
    internal string bank_id { get; set; }
    internal string amount { get; set; }
    internal string company_order_num { get; set; }
    internal string company_user { get; set; }
    internal string estimated_payment_bank { get; set; }
    internal string deposit_mode { get; set; }
    internal string group_id { get; set; }
    internal string web_url { get; set; }
    internal string memo { get; set; }
    internal string key { get; set; }
    internal string terminal { get; set; }
    internal JuBaoPenPayDepositSubmissionRequest(
            string strcompany_id,
            string strbank_id,
            string stramount,
            string strcompany_order_num,
            string strcompany_user,
            string strestimated_payment_bank,
            string strdeposit_mode,
            string strgroup_id,
            string strweb_url,
            string strmemo,
            string strkey,
            string strterminal)
    {
        company_id = strcompany_id;
        bank_id = strbank_id;
        amount = stramount;
        company_order_num = strcompany_order_num;
        company_user = strcompany_user;
        estimated_payment_bank = strestimated_payment_bank;
        deposit_mode = strdeposit_mode;
        group_id = strgroup_id;
        web_url = strweb_url;
        memo = strmemo;
        key = strkey;
        terminal = strterminal;
    }
}

In [ ]:
#region Helper
    internal static class Helper
    {
        internal static string Base64Encode(string plainText)
        {
            var plainTextBytes = Encoding.UTF8.GetBytes(plainText);
            return Convert.ToBase64String(plainTextBytes);
        }

        internal static string Base64Decode(string base64EncodedData)
        {
            var base64EncodedBytes = Convert.FromBase64String(base64EncodedData);
            return Encoding.UTF8.GetString(base64EncodedBytes);
        }

        /// <summary>
        /// Generate encrypted key in base64
        /// </summary>
        /// <param name="strKey1">Data to be stored in</param>
        /// <returns></returns>
        internal static string GenerateKey(string strKey1)
        {
            string encryptedText = Base64Encode(strKey1);
            return encryptedText;
        }

        internal static string Stringify(object obj)
        {
            try
            {
                if (obj == null)
                    return string.Empty;
                if (obj is decimal)
                    return ((decimal)obj).ToString();
                if (obj is int?)
                    return ((int?)obj).HasValue ? obj.ToString() : string.Empty;
                return obj.ToString().Trim();
            }
            catch (Exception ex)
            {
                throw;
            }
        }

        /// <summary>
        /// Signature Generations
        /// </summary>
        internal static string SignatureGeneration(Dictionary<string, string> requestDataDict,
            Dictionary<string, string> dictKeys,
            JuBaoPenPayActionType actionType)
        {
            string message = string.Empty;
            string signature = string.Empty;

            switch (actionType)
            {
                case JuBaoPenPayActionType.DepositRequest:
                    {
                        message = HashMessage(requestDataDict, dictKeys, true, false, false, false);
                        signature = AppCryptoHelper.GetMd5Hash(message).ToLower();
                        break;
                    }
                case JuBaoPenPayActionType.DepositNotify:
                    {
                        message = HashMessage(requestDataDict, dictKeys, false, true, false, false);
                        signature = AppCryptoHelper.GetMd5Hash(message).ToLower();
                        break;
                    }
                case JuBaoPenPayActionType.WithdrawalRequest:
                    {
                        message = HashMessage(requestDataDict, dictKeys, false, false, true, false);
                        signature = AppCryptoHelper.GetMd5Hash(message).ToLower();
                        break;
                    }

                case JuBaoPenPayActionType.WithdrawalNotify:
                    {
                        message = HashMessage(requestDataDict, dictKeys, false, false, false, true);
                        signature = AppCryptoHelper.GetMd5Hash(message).ToLower();
                        break;
                    }
                default:
                    {
                        signature = AppCryptoHelper.GetMd5Hash(message).ToLower();
                        break;
                    }
            }

            return signature;
        }
        /// <summary>
        /// Hash Message Generations
        /// </summary>
        internal static string HashMessage(Dictionary<string, string> requestDataDict,
            Dictionary<string, string> dictKeys,
            bool depositRequest, bool depositNotify,
            bool withdrawalRequest, bool withdrawalNotify)
        {
            string message = string.Empty;
            string message2 = string.Empty;
            string message3 = string.Empty;
            string message4 = string.Empty;
            int i = 0;
            foreach (KeyValuePair<string, string> value in requestDataDict)
            {
                if (!String.IsNullOrEmpty(value.Value))
                {
                    if (depositRequest)
                    {
                        if (Enum.TryParse(value.Key, out JuBaoPenPayDepositMessageRequest checkEnum))
                        {
                            message += $"{value.Value}";
                        }
                    }
                    if (depositNotify)
                    {
                        if (Enum.TryParse(value.Key, out JuBaoPenPayDepositNotifyRequest checkEnum))
                        {
                            message += $"{value.Value}";
                        }
                        if (Enum.TryParse(value.Key, out JuBaoPenPayDepositNotifyRequest2 checkEnum2))
                        {
                            message2 += $"{value.Value}";
                        }
                    }
                    if (withdrawalRequest)
                    {
                        if (Enum.TryParse(value.Key, out JuBaoPenPayWithdrawalMessageRequest checkEnum))
                        {
                            message += $"{value.Value}";
                        }
                        if (Enum.TryParse(value.Key, out JuBaoPenPayWithdrawalMessageRequest2 checkEnum2))
                        {
                            message2 += $"{value.Value}";
                        }
                        if (Enum.TryParse(value.Key, out JuBaoPenPayWithdrawalMessageRequest3 checkEnum3))
                        {
                            message3 += $"{value.Value}";
                        }
                        if (Enum.TryParse(value.Key, out JuBaoPenPayWithdrawalMessageRequest4 checkEnum4))
                        {
                            message4 += $"{value.Value}";
                        }
                    }
                    if (withdrawalNotify)
                    {
                        if (Enum.TryParse(value.Key, out JuBaoPenPayWithdrawalNotifyRequest checkEnum))
                        {
                            message += $"{value.Value}";
                        }

                        if (Enum.TryParse(value.Key, out JuBaoPenPayWithdrawalNotifyRequest2 checkEnum2))
                        {
                            message2 += $"{value.Value}";
                        }

                        if (Enum.TryParse(value.Key, out JuBaoPenPayWithdrawalNotifyRequest3 checkEnum3))
                        {
                            message3 += $"{value.Value}";
                        }
                    }
                }
                i++;
            }
            message = message + message2 + message3 + message4;
            var key = dictKeys.ContainsKey(JuBaoPenPaySecureKey.Key.ToString())
                ? AppCryptoHelper.GetMd5Hash($"{dictKeys[JuBaoPenPaySecureKey.Key.ToString()]}").ToLower()
                : string.Empty;
            message = key+message;

            return message;
        }

        internal static Dictionary<string, string> StatusCode = new Dictionary<string, string>
        {
            { "0", "订单生成" },
            { "1", "支付中" },
            { "2", "支付成功" },
            { "3", "业务处理完成" },
            { "5", " 支付失败" }
        };
    }
    #endregion Helper

In [ ]:
#region Enum
    internal enum JuBaoPenPaySecureKey
    {
        Key
    }

    internal enum JuBaoPenPayDepositMessageRequest
    {
        company_id,
        bank_id,
        amount,
        company_order_num,
        company_user,
        estimated_payment_bank,
        deposit_mode,
        group_id,
        web_url,
        memo,
        note,
        note_model
    }


    internal enum JuBaoPenPayDepositNotifyRequest
    {
        pay_time,
        bank_id,
        amount,
        company_order_num,
        mownecum_order_num,
        fee
    }

    internal enum JuBaoPenPayDepositNotifyRequest2
    {
        transaction_charge,
        deposit_mode
    }


    internal enum JuBaoPenPayWithdrawalMessageRequest
    {
        company_id,
        bank_id,
        company_order_num
    }
    internal enum JuBaoPenPayWithdrawalMessageRequest2
    {
        amount,
        card_num,
        card_name
    }
    internal enum JuBaoPenPayWithdrawalMessageRequest3
    {
        company_user,
        web_url
    }
    internal enum JuBaoPenPayWithdrawalMessageRequest4
    {
        issue_bank_name,
        issue_bank_address,
        memo
    }

    internal enum JuBaoPenPayWithdrawalNotifyRequest
    {
        mownecum_order_num
    }

    internal enum JuBaoPenPayWithdrawalNotifyRequest2
    {
        company_order_num
    }

    internal enum JuBaoPenPayWithdrawalNotifyRequest3
    {
        status,
        amount,
        exact_transaction_charge
    }


    internal enum JuBaoPenPayActionType
    {
        DepositRequest,
        WithdrawalRequest,
        DepositResponse,
        WithdrawalResponse,
        DepositNotify,
        WithdrawalNotify
    }
#endregion Enum

In [ ]:
#region Crypto

    using System;
    using System.IO;
    using System.Security.Cryptography;
    using System.Text;

    public class AppCryptoHelper
    {
        /// <summary>
        /// use AES 256 to encrypt
        /// </summary>
        /// <param name="source">context</param>
        /// <param name="key">32 chars => 32*8=256</param>
        /// <param name="iv">16 chars => 16 * 8= 128</param>
        /// <returns></returns>

        public static string EncryptAES256(string source, string key, string iv)
        {
            byte[] sourceBytes = Encoding.UTF8.GetBytes(source);
            var aes = new RijndaelManaged();
            aes.Key = Encoding.UTF8.GetBytes(key);
            aes.IV = Encoding.UTF8.GetBytes(iv);
            aes.Mode = CipherMode.CBC;
            aes.Padding = PaddingMode.PKCS7;
            ICryptoTransform transform = aes.CreateEncryptor();
            return Convert.ToBase64String(transform.TransformFinalBlock(sourceBytes, 0, sourceBytes.Length));

        }

        /// <summary>
        /// use AES 256 to encrypt
        /// </summary>
        /// <param name="encryptData">encrypted by Base64</param>
        /// <param name="key">32 chars => 32*8=256</param>
        /// <param name="iv">16 chars => 16 * 8= 128</param>
        /// <returns></returns>
        public static string DecryptAES256(string encryptData, string key, string iv)
        {
            var encryptBytes = Convert.FromBase64String(encryptData);
            var aes = new RijndaelManaged();
            aes.Key = Encoding.UTF8.GetBytes(key);
            aes.IV = Encoding.UTF8.GetBytes(iv);

            aes.Mode = CipherMode.CBC;
            aes.Padding = PaddingMode.PKCS7;
            ICryptoTransform transform = aes.CreateDecryptor();
            return Encoding.UTF8.GetString(transform.TransformFinalBlock(encryptBytes, 0, encryptBytes.Length));
        }
        public static string EnpryptAES(string plainText, string aeskey, string ivkey)
        {
            var AES = new RijndaelManaged();
            byte[] plainTextData = Encoding.UTF8.GetBytes(plainText);
            byte[] keyData = Encoding.UTF8.GetBytes(aeskey);
            byte[] IVData = Encoding.UTF8.GetBytes(ivkey);
            ICryptoTransform transform = AES.CreateEncryptor(keyData, IVData);
            byte[] outputData = transform.TransformFinalBlock(plainTextData, 0, plainTextData.Length);
            return Convert.ToBase64String(outputData);
        }
        public static string DecryptAES(string plainText, string aeskey, string ivkey)
        {
            var AES = new RijndaelManaged();
            var cipherTextData = Convert.FromBase64String(plainText);
            byte[] keyData = Encoding.UTF8.GetBytes(aeskey);
            byte[] IVData = Encoding.UTF8.GetBytes(ivkey);
            ICryptoTransform transform = AES.CreateDecryptor(keyData, IVData);
            byte[] outputData = transform.TransformFinalBlock(cipherTextData, 0, cipherTextData.Length);
            return Encoding.UTF8.GetString(outputData);
        }
        public static string GetMd5Hash(string input) => GetMd5Hash(input, Encoding.UTF8);
        public static string GetMd5Hash(string input, Encoding encoding)
        {
            StringBuilder sBuilder = new StringBuilder();
            using (MD5 md5Hash = MD5.Create())
            {
                byte[] data = md5Hash.ComputeHash(encoding.GetBytes(input));
                for (int i = 0; i < data.Length; i++)
                    sBuilder.Append(data[i].ToString("x2"));
            }
            return sBuilder.ToString();
        }

        public static string EncryptSHA256(string input)
        {
            using (var sha256 = new SHA256Managed())
            {
                return BitConverter.ToString(sha256.ComputeHash(Encoding.UTF8.GetBytes(input))).Replace("-", string.Empty);
            }
        }

        public static string EncryptSHA256WithBase64(string plainText)
        {
            using (var sha256 = new SHA256Managed())
            {
               return System.Convert.ToBase64String(sha256.ComputeHash(Encoding.UTF8.GetBytes(plainText))).Replace("-", string.Empty);
            }
        }

        public static string GetHMACSHA256(string plainText, string shakey) => GetHMACSHA256(plainText, shakey, Encoding.UTF8);

        public static string GetHMACSHA256(string plainText, string shakey, Encoding encoding)
        {
            byte[] encodedKey = encoding.GetBytes(shakey);
            byte[] encodedPlainText = encoding.GetBytes(plainText);

            byte[] hashResult;

            HMACSHA256 hmac = new HMACSHA256(encodedKey);

            hashResult = hmac.ComputeHash(encodedPlainText);
            string digest = BitConverter.ToString(hashResult).Replace("-", "");
            hmac.Dispose();

            return digest;
        }

        public static string EncryptDES_ToHEX(string plainText, string sKey, Encoding encoding)
        {
            // Create a MemoryStream.
            System.IO.MemoryStream mStream = new System.IO.MemoryStream();

            // Create a new DES object.
            DES DESalg = DES.Create();
            //Set secret key For DES algorithm.
            DESalg.Key = encoding.GetBytes(sKey);
            //Set initialization vector.
            DESalg.IV = encoding.GetBytes(sKey);

            // Create a CryptoStream using the MemoryStream
            CryptoStream cStream = new CryptoStream(mStream, DESalg.CreateEncryptor(), CryptoStreamMode.Write);

            // Convert the passed string to a byte array.
            byte[] toEncrypt = encoding.GetBytes(plainText);

            // Write the byte array to the crypto stream and flush it.
            cStream.Write(toEncrypt, 0, toEncrypt.Length);
            cStream.FlushFinalBlock();

            // Get an array of bytes from the
            // MemoryStream that holds the
            // encrypted data.
            byte[] ret = mStream.ToArray();

            // Close the streams.
            cStream.Close();
            mStream.Close();

            // Convert Byte to Hex String
            StringBuilder hex = new StringBuilder(ret.Length * 2);
            foreach (byte b in ret)
                hex.AppendFormat("{0:x2}", b);

            return hex.ToString();
        }

        public static string EncryptDES_ToHEX(string plainText, string sKey, byte[] IV, CipherMode cipherMode, Encoding encoding)
        {
            // Create a MemoryStream.
            System.IO.MemoryStream mStream = new System.IO.MemoryStream();

            // Create a new DES object.
            DES DESalg = DES.Create();
            //Set cipher mode For DES algorithm.
            DESalg.Mode = cipherMode;
            //Set secret key For DES algorithm.
            DESalg.Key = encoding.GetBytes(sKey);
            //Set initialization vector.
            DESalg.IV = IV.Length > 0 ? IV : encoding.GetBytes(sKey);

            // Create a CryptoStream using the MemoryStream
            CryptoStream cStream = new CryptoStream(mStream, DESalg.CreateEncryptor(), CryptoStreamMode.Write);

            // Convert the passed string to a byte array.
            byte[] toEncrypt = encoding.GetBytes(plainText);

            // Write the byte array to the crypto stream and flush it.
            cStream.Write(toEncrypt, 0, toEncrypt.Length);
            cStream.FlushFinalBlock();

            // Get an array of bytes from the
            // MemoryStream that holds the
            // encrypted data.
            byte[] ret = mStream.ToArray();

            // Close the streams.
            cStream.Close();
            mStream.Close();

            // Convert Byte to Hex String
            StringBuilder hex = new StringBuilder(ret.Length * 2);
            foreach (byte b in ret)
                hex.AppendFormat("{0:x2}", b);

            return hex.ToString();
        }

        public static string DecryptDES_FromHEX(string cipherText, string sKey, Encoding encoding)
        {
            // Create a new DES object.
            DES DESalg = DES.Create();
            //Set secret key For DES algorithm.
            DESalg.Key = encoding.GetBytes(sKey);
            //Set initialization vector.
            DESalg.IV = encoding.GetBytes(sKey);

            // Convert Hex String to Byte
            int NumberChars = cipherText.Length;
            byte[] bytes = new byte[NumberChars / 2];
            for (int i = 0; i < NumberChars; i += 2)
                bytes[i / 2] = Convert.ToByte(cipherText.Substring(i, 2), 16);

            // Create a MemoryStream.
            System.IO.MemoryStream mStream = new System.IO.MemoryStream(bytes);

            // Create a CryptoStream using the FileStream
            CryptoStream cStream = new CryptoStream(mStream, DESalg.CreateDecryptor(), CryptoStreamMode.Read);

            // Create a StreamReader using the CryptoStream.
            System.IO.StreamReader sReader = new System.IO.StreamReader(cStream);

            // Read the data from the stream
            // to decrypt it.
            string plainText = sReader.ReadLine();

            // Close the streams and
            // close the file.
            sReader.Close();
            cStream.Close();
            mStream.Close();

            return plainText;
        }

        public static string DecryptDES_FromHEX(string cipherText, string sKey, byte[] IV, CipherMode cipherMode, Encoding encoding)
        {
            // Create a new DES object.
            DES DESalg = DES.Create();
            //Set cipher mode For DES algorithm.
            DESalg.Mode = cipherMode;
            //Set secret key For DES algorithm.
            DESalg.Key = encoding.GetBytes(sKey);
            //Set initialization vector.
            DESalg.IV = IV.Length > 0 ? IV : encoding.GetBytes(sKey);

            // Convert Hex String to Byte
            int NumberChars = cipherText.Length;
            byte[] bytes = new byte[NumberChars / 2];
            for (int i = 0; i < NumberChars; i += 2)
                bytes[i / 2] = Convert.ToByte(cipherText.Substring(i, 2), 16);

            // Create a MemoryStream.
            System.IO.MemoryStream mStream = new System.IO.MemoryStream(bytes);

            // Create a CryptoStream using the FileStream
            CryptoStream cStream = new CryptoStream(mStream, DESalg.CreateDecryptor(), CryptoStreamMode.Read);

            // Create a StreamReader using the CryptoStream.
            System.IO.StreamReader sReader = new System.IO.StreamReader(cStream);

            // Read the data from the stream
            // to decrypt it.
            string plainText = sReader.ReadLine();

            // Close the streams and
            // close the file.
            sReader.Close();
            cStream.Close();
            mStream.Close();

            return plainText;
        }

        public static string EncryptDES_ToBase64String(string plainText, string Key, byte[] IV, CipherMode cipherMode, Encoding encoding)
        {
            byte[] byKey = null;
            byte[] byIV = null;

            byKey = encoding.GetBytes(Key.Substring(0, Key.Length));
            byIV = IV.Length > 0 ? IV : encoding.GetBytes(Key.Substring(0, Key.Length));
            System.Security.Cryptography.DESCryptoServiceProvider des = new System.Security.Cryptography.DESCryptoServiceProvider();
            byte[] inputByteArray = encoding.GetBytes(plainText);
            System.IO.MemoryStream ms = new System.IO.MemoryStream();
            System.Security.Cryptography.CryptoStream cs = new System.Security.Cryptography.CryptoStream(ms, des.CreateEncryptor(byKey, byIV), System.Security.Cryptography.CryptoStreamMode.Write);
            cs.Write(inputByteArray, 0, inputByteArray.Length);
            cs.FlushFinalBlock();
            return Convert.ToBase64String(ms.ToArray());
        }

        public static string DecryptDES_FromBase64String(string cipherText, string Key, byte[] IV, CipherMode cipherMode, Encoding encoding)
        {
            byte[] byKey = null;
            byte[] byIV = null;
            byte[] inputByteArray = new Byte[cipherText.Length];

            byKey = encoding.GetBytes(Key.Substring(0, Key.Length));
            byIV = IV.Length > 0 ? IV : encoding.GetBytes(Key.Substring(0, Key.Length));
            System.Security.Cryptography.DESCryptoServiceProvider des = new System.Security.Cryptography.DESCryptoServiceProvider();
            inputByteArray = Convert.FromBase64String(cipherText);
            System.IO.MemoryStream ms = new System.IO.MemoryStream();
            System.Security.Cryptography.CryptoStream cs = new System.Security.Cryptography.CryptoStream(ms, des.CreateDecryptor(byKey, byIV), System.Security.Cryptography.CryptoStreamMode.Write);
            cs.Write(inputByteArray, 0, inputByteArray.Length);
            cs.FlushFinalBlock();
            return encoding.GetString(ms.ToArray());
        }

        public static string EncryptRSA(string xmlPublicKey, byte[] plainbytes)
        {
            using (RSACryptoServiceProvider rsa = new RSACryptoServiceProvider())
            {
                rsa.FromXmlString(xmlPublicKey);
                //rsa.FromXmlString(Encoding.UTF8.GetString(Convert.FromBase64String(publicKey)));
                var bufferSize = (rsa.KeySize / 8 - 11);
                byte[] buffer = new byte[bufferSize];//待加密块

                using (MemoryStream msInput = new MemoryStream(plainbytes))
                {
                    using (MemoryStream msOutput = new MemoryStream())
                    {
                        int readLen;
                        while ((readLen = msInput.Read(buffer, 0, bufferSize)) > 0)
                        {
                            byte[] dataToEnc = new byte[readLen];
                            Array.Copy(buffer, 0, dataToEnc, 0, readLen);
                            byte[] encData = rsa.Encrypt(dataToEnc, false);
                            msOutput.Write(encData, 0, encData.Length);
                        }

                        byte[] result = msOutput.ToArray();
                        rsa.Clear();
                        return Convert.ToBase64String(result);
                    }
                }
            }
        }

        public static string DecryptRSA(string xmlPrivateKey, byte[] cipherbytes)
        {
            using (RSACryptoServiceProvider rsa = new RSACryptoServiceProvider())
            {
                rsa.FromXmlString(xmlPrivateKey);
                //rsa.FromXmlString(Encoding.UTF8.GetString(Convert.FromBase64String(privateKey)));
                int keySize = rsa.KeySize / 8;
                byte[] buffer = new byte[keySize];
                using (MemoryStream msInput = new MemoryStream(cipherbytes))
                {
                    using (MemoryStream msOutput = new MemoryStream())
                    {
                        int readLen;

                        while ((readLen = msInput.Read(buffer, 0, keySize)) > 0)
                        {
                            byte[] dataToDec = new byte[readLen];
                            Array.Copy(buffer, 0, dataToDec, 0, readLen);
                            byte[] decData = rsa.Decrypt(dataToDec, false);
                            msOutput.Write(decData, 0, decData.Length);
                        }

                        byte[] result = msOutput.ToArray();
                        rsa.Clear();

                        return Encoding.UTF8.GetString(result);
                    }
                }
            }
        }
    }

#endregion Crypto

In [ ]:
using System;
using System.Collections.Generic;
using System.ComponentModel;
using System.Data;
using System.Linq;
using System.Linq.Expressions;
using System.Reflection;
public static class ReflectionHelper
{
    public static string GetPropertyName<T>(Expression<Func<T>> propertyLambda)
    {
        var me = propertyLambda.Body as MemberExpression;

        if (me == null)
        {
            throw new ArgumentException("You must pass a lambda of the form: '() => Class.Property' or '() => object.Property'");
        }

        return me.Member.Name;
    }

    public static DataTable ToDataTable<T>(IList<T> data)
    {
        PropertyDescriptorCollection props =
            TypeDescriptor.GetProperties(typeof(T));
        DataTable table = new DataTable();
        for (int i = 0; i < props.Count; i++)
        {
            PropertyDescriptor prop = props[i];
            table.Columns.Add(prop.Name, prop.PropertyType);
        }
        object[] values = new object[props.Count];
        foreach (T item in data)
        {
            for (int i = 0; i < values.Length; i++)
            {
                values[i] = props[i].GetValue(item);
            }
            table.Rows.Add(values);
        }
        return table;
    }
    /// <summary>
    /// Convert Object to dictionary
    /// </summary>
    /// <typeparam name="Tin">Data type Object to Convert</typeparam>
    /// <typeparam name="Tout">data type for value from string key</typeparam>
    /// <param name="obj">Object to convert</param>
    /// <param name="bindingFlags">reflection Binding Flags</param>
    /// <param name="Query">set it to null to get all property, use linq to filter</param>
    /// <param name="translator">function to convert each property to desired data type for value</param>
    /// <returns></returns>
    public static Dictionary<string, Tout> AsDictionary<Tin, Tout>(Tin obj, System.Reflection.BindingFlags bindingFlags = BindingFlags.Public, Func<PropertyInfo[], IEnumerable<PropertyInfo>> Query = null, Func<object,Tout> translator = null)
    {
        var props = obj.GetType().GetProperties(bindingFlags);
        var propsRules = Query?.Invoke(props) ?? props;
        return propsRules
            .ToDictionary(ky => ky.Name, val => translator != null ? translator.Invoke(val.GetValue(obj, null)) : (Tout)val.GetValue(obj, null));
    }
}

In [ ]:
string a = "12345678-1234-1234-1234-123456781234";

In [ ]:
Guid guid = new Guid(a);

In [ ]:
DateTime currentDateTime = DateTime.Now;
string clientIP = "::1";
byte[] tokenbytes = guid.ToByteArray();
byte[] dtbytes = BitConverter.GetBytes(currentDateTime.ToBinary());


In [ ]:
using System.Net;
public static byte[] ConvertIPToBytes(string ip)
{
    IPAddress ipaddress;
    bool isValid = IPAddress.TryParse(ip, out ipaddress);

    if (isValid)
    {
        return ipaddress.GetAddressBytes();
    }
    return null;
}



In [ ]:
byte[] ipbytes = ConvertIPToBytes(clientIP);
byte[] msg = new byte[tokenbytes.Length + dtbytes.Length + ipbytes.Length];

In [ ]:
tokenbytes.CopyTo(msg, 0);
dtbytes.CopyTo(msg, tokenbytes.Length);
ipbytes.CopyTo(msg, tokenbytes.Length + dtbytes.Length);

In [ ]:

public class resCampaignBaseParam
{
    public string Message { get; set; }
}

public class resWithStatus : resCampaignBaseParam
{
    public bool Success { get; set; }
}

In [ ]:
resWithStatus result = new resWithStatus{Message = "yr", Success = true};

In [ ]:
using Newtonsoft.Json;

In [ ]:
var jsonrequest = JsonConvert.SerializeObject(result);

In [ ]:
public class test
{
    public bool Success { get; set; }
    public string Message { get; set; }
}

In [ ]:
test result3 = new test{Success = true, Message = "yr"};

In [ ]:
result3

In [ ]:
resWithStatus result4 = new resWithStatus();
result4 = JsonConvert.DeserializeObject<resWithStatus>(jsonrequest);

In [ ]:
result4

In [ ]:
public class test
{
    public bool? Success { get; set; }
    public string Message { get; set; }
}
public class test2
{
    public bool fail { get; set; }
    public string fm { get; set; }
}

In [ ]:
var t = new test{
    Success = null,
    Message = "test"
};
var t2 = new test2{
    fail = false,
    fm = "test2"
};

In [ ]:
public TResponse GetSlotCurrentDraw<TResponse>(string a)
{
    Console.WriteLine(typeof(TResponse).ToString());
    switch (typeof(TResponse))
    {
        case Type t when t == typeof(test) :
            Console.WriteLine(a);
            break;

        case Type t when t == typeof(test2):
            Console.WriteLine("qwe");
            break;
        default:
            // Default process for other types
            Console.WriteLine("Processing for other types");
            break;
    }
    return default(TResponse);
}


In [ ]:
var price = GetSlotCurrentDraw<int>("123");

In [ ]:
private bool _checkCampaignValid<TResponse>(TResponse spiResponse)
{
    bool isValid = false;
    if(typeof(TResponse) == typeof(test))
    {
            isValid = true;
    }

    if(typeof(TResponse) == typeof(test2))
    {
            isValid = false;
    }
    return isValid;

}

In [ ]:
_checkCampaignValid<test>(t)

In [ ]:
_checkCampaignValid<test2>(t2)

In [ ]:
t.Success.HasValue

In [ ]:
public enum enumCampaignType
    {
        HongBao = 1,
        STW = 2,
        Slot = 3
    }

In [ ]:
int campaignType = 0;

In [ ]:
campaignType = enumCampaignType.STW;

In [ ]:
enumCampaignType campaignType = enumCampaignType.STW;

In [ ]:
campaignType

In [ ]:
if(campaignType == enumCampaignType.STW){
    Console.WriteLine("True");
}

In [ ]:
bool t4 = t != null? true : false;

In [ ]:
test t3=null;

In [ ]:
t4 ? true : false

In [ ]:
private TResponse _getPrizeAPI<TRequest, TResponse>(TRequest request)
{
    Console.WriteLine(request.ToString());
    var result  = DateTime.Now;
    return result;
}

In [ ]:
if(campaignType is enumCampaignType.STW){
    Console.WriteLine("True");
}

In [ ]:
public string _getTncUrl<TResponse>(enumPlatformType platformType, TResponse spiResponse, enumCampaignType campaignType)
        {
            string promotionUrl = null;

            switch (campaignType)
            {
                case enumCampaignType.Slot:
                    promotionUrl = (platformType == enumPlatformType.APP && promotionUrl == null) 
                        ? ((test)(object)spiResponse).Success.ToString() : promotionUrl;
                    promotionUrl = (platformType == enumPlatformType.Desktop && promotionUrl == null)
                        ? ((test)(object)spiResponse).Message : promotionUrl;
                    promotionUrl = (platformType == enumPlatformType.H5 && promotionUrl == null)
                        ? ((test)(object)spiResponse).Success.ToString() : promotionUrl;
                    break;
                default:
                    break;
            }
            return promotionUrl;
        }


In [ ]:
public enum enumPlatformType
    {
        APP = 1,
        Desktop = 2,
        H5 = 3
    }

In [ ]:
var re =_getTncUrl<test>(enumPlatformType.Desktop, t, enumCampaignType.Slot);

In [ ]:
t.Success.HasValue

In [ ]:
public class test3
{
    public bool Success { get; set; }
    public test? Message { get; set; }
}

In [ ]:
var t30 = new test3 {
    Success = true,
    Message = new test{
        Success = true,
        Message = "test"
    }
};

In [ ]:
t30.Message.HasValue

In [ ]:
private TResponse RejectSlotPrize<TResponse>(string a)
{
    TResponse result = default(TResponse);
    if(typeof(TResponse) == typeof(test2)){
        var t2 = new test2{
            fail = false,
            fm = a
        };
        result = (TResponse)(object)t2;

    }

    if(typeof(TResponse) == typeof(test)){
        var t2 = new test{
            Success = true,
            Message= a
        };
        result = (TResponse)(object)t2;

    }
    
    return result;
}

In [ ]:
var bg = RejectSlotPrize<test2>("abc");

In [ ]:
Console.WriteLine(bg.GetType().ToString());

In [ ]:
Console.WriteLine(bg.GetType().ToString());

In [ ]:
Console.WriteLine("test");

In [ ]:
private int RejectSlotPrize(string a)
{
    var result = 2;
    
    return result;
}

In [ ]:
RejectSlotPrize("s")

In [ ]:
public class MyObject
{
    public int Id { get; set; }
    public decimal? Amt { get; set; }
    public int ValueToSum { get; set; }
    public int? RemainCount { get; set; }
    public int GivenCount 
    { 
        get { return ValueToSum - (int)(RemainCount ?? 0); }         
    }
}
List<MyObject> myList = new List<MyObject>
{
    new MyObject { Id = 1, Amt = 20.00M, ValueToSum = 5, RemainCount = 2 },
    new MyObject { Id = 1, Amt = 20.00M, ValueToSum = 5, RemainCount = 2 },
    new MyObject { Id = 3, Amt = 5.00M, ValueToSum = 10, RemainCount = 3 }
};


In [ ]:
myList = null;
myList.Count

In [ ]:
var x = myList.GroupBy(g =>new { g.Id, g.Amt, g.ValueToSum, g.RemainCount, g.GivenCount }).ToList();
dynamic(x)
x[0].Key.Amt

In [ ]:
x[0].Key.Amt

In [ ]:
dynamic(x)

In [ ]:
myList.Where(x=>x.Amt.HasValue).Where(x=>!string.IsNullOrEmpty(x.Amt.ToString())).Select(x => x.Amt).ToList()

In [ ]:
*27*322

In [ ]:
reqElligibleStakeMdl requestData = new reqElligibleStakeMdl
        {
            Stake = new List<ElligibleStakeModel>
            {
                new ElligibleStakeModel { USDT = 100, RMB = 500, VIP1 = 10, VIP2 = 20, VIP3 = 30, VIP4 = 40, VIP5 = 15, VIP6 = 25, VIP7 = 35, VIP8 = 45 },
                new ElligibleStakeModel { USDT = 150, RMB = 600, VIP1 = 25, VIP2 = 30, VIP3 = 35, VIP4 = 45, VIP5 = 20, VIP6 = 30, VIP7 = 40, VIP8 = 50 },
            }
        };

      



In [ ]:
 public class reqElligibleStakeMdl
{
    public List<ElligibleStakeModel> Stake { get; set; }

    public bool ValidateStake()
    {
        bool result = false;
        result = ValidateMaxFreeSpin() && CheckAscendingOrder() && CheckElligibleStakeValue();
        return result;
    }

    private bool ValidateMaxFreeSpin()
    {
        bool result = false;
        List<List<bool>> results = new List<List<bool>>();
        for (int i = 0; i < 8; i++)
        {
            int vipIndex = i + 1 ;
            results.Add(Stake.Select(x => x.GetType().GetProperty($"VIP{vipIndex}").GetValue(x) as int? > 15).ToList());
        }
        result = results.Any(x => x.Contains(true)) ? false : true;
        return result;
    }

    private bool CheckAscendingOrder()
    {
        int numRows = 8;
        int numCols = Stake.Count;

        for (int col = 0; col < numCols; col++)
        {
            int lastNonZeroIndex = -1;

            // Find the index of the last non-zero value in the current column
            for (int row = 0; row < numRows; row++)
            {
                if (Stake[col].GetVIPValue(row + 1 ) != 0)
                {
                    lastNonZeroIndex = row + 1;
                }
            }

            // Check if each value in the column is greater than or equal to the previous row until the last non-zero value
            for (int row = 2; row <= lastNonZeroIndex; row++)
            {
                if (Stake[col].GetVIPValue(row) < Stake[col].GetVIPValue(row-1))
                {
                    return false; // Return false as soon as a column is found not satisfying the condition
                }
            }
        }

        return true; // If no column is found not satisfying the condition, return true
    }

    private bool CheckElligibleStakeValue(){
        bool result =  false;
        var usdtList = Stake.Select(x=>x.USDT).ToList();
        var rmbList = Stake.Select(x=>x.RMB).ToList();
        result = rmbList.SequenceEqual(rmbList.OrderBy(x => x)) 
                    && rmbList.Distinct().Count() == rmbList.Count 
                    && usdtList.SequenceEqual(usdtList.OrderBy(x => x)) 
                    && usdtList.Distinct().Count() == usdtList.Count;

        return result;
    }
}

public class ElligibleStakeModel2
{
    public int VIP1 { get; set; }
    public int VIP2 { get; set; }
    public int VIP3 { get; set; }
    public int VIP4 { get; set; }
    public int VIP5 { get; set; }
    public int VIP6 { get; set; }
    public int VIP7 { get; set; }
    public int VIP8 { get; set; }
    public int USDT { get; set; }
    public int RMB { get; set; }

    public decimal GetVIPValue(int vipColumn)
    {
        switch (vipColumn)
        {
            case 1: return VIP1;
            case 2: return VIP2;
            case 3: return VIP3;
            case 4: return VIP4;
            case 5: return VIP5;
            case 6: return VIP6;
            case 7: return VIP7;
            case 8: return VIP8;
            default: throw new ArgumentOutOfRangeException(nameof(vipColumn), "Invalid VIP column number");
        }
    }

    public int GetCurrencyValue(string currency)
    {
        switch (currency)
        {
            case "USDT": return USDT;
            case "RMB" : return RMB;
            default: throw new ArgumentOutOfRangeException(nameof(currency), "Invalid currency code.");
        }
    }

    
}

In [ ]:
public class reqElligibleStakeMdl
    {
        public List<ElligibleStakeModel> FreeSpin { get; set; }

        public bool ValidateStake()
        {
            bool result = false;
            result = ValidateMaxFreeSpin() && CheckAscendingOrder() && CheckElligibleStakeValue();
            return result;
        }

        private bool ValidateMaxFreeSpin()
        {
            bool result = false;
            List<List<bool>> results = new List<List<bool>>();
            for (int i = 0; i < 8; i++)
            {
                int vipIndex = i + 1;
                results.Add(FreeSpin.Select(x => x.GetType().GetProperty($"Vip{vipIndex}").GetValue(x) as int? > 15).ToList());
            }
            result = results.Any(x => x.Contains(true)) ? false : true;
            return result;
        }

        private bool CheckAscendingOrder()
        {
            int numRows = 8;
            int numCols = FreeSpin.Count;

            for (int col = 0; col < numCols; col++)
            {
                int lastNonZeroIndex = -1;

                for (int row = 0; row < numRows; row++)
                {
                    if (FreeSpin[col].GetVIPValue(row + 1) != 0)
                    {
                        lastNonZeroIndex = row + 1;
                    }
                }

                for (int row = 2; row <= lastNonZeroIndex; row++)
                {
                    if (FreeSpin[col].GetVIPValue(row) < FreeSpin[col].GetVIPValue(row - 1))
                    {
                        return false; 
                    }
                }
            }

            return true; 
        }

        private bool CheckElligibleStakeValue()
        {
            bool result = false;
            var usdtList = FreeSpin.Select(x => x.StakeUsdt).ToList();
            var rmbList = FreeSpin.Select(x => x.StakeRmb).ToList();
            result = rmbList.SequenceEqual(rmbList.OrderBy(x => x))
                        && rmbList.Distinct().Count() == rmbList.Count
                        && usdtList.SequenceEqual(usdtList.OrderBy(x => x))
                        && usdtList.Distinct().Count() == usdtList.Count;

            return result;
        }
    }

    public class ElligibleStakeModel
    {
        //todo add new draw model
        public int StakeUsdt { get; set; }
        public int StakeRmb { get; set; }
        public int Vip1 { get; set; }
        public int Vip2 { get; set; }
        public int Vip3 { get; set; }
        public int Vip4 { get; set; }
        public int Vip5 { get; set; }
        public int Vip6 { get; set; }
        public int Vip7 { get; set; }
        public int Vip8 { get; set; }

        public decimal GetVIPValue(int vipColumn)
        {
            switch (vipColumn)
            {
                case 1: return Vip1;
                case 2: return Vip2;
                case 3: return Vip3;
                case 4: return Vip4;
                case 5: return Vip5;
                case 6: return Vip6;
                case 7: return Vip7;
                case 8: return Vip8;
                default: throw new ArgumentOutOfRangeException(nameof(vipColumn), "Invalid VIP column number");
            }
        }

    }

In [ ]:
var Stake = new reqElligibleStakeMdl {
    FreeSpin = new List<ElligibleStakeModel>
    {
        new ElligibleStakeModel { StakeUsdt = 150, StakeRmb = 500, Vip1 = 1, Vip2 = 2, Vip3 = 4, Vip4 = 5, Vip5 = 0, Vip6 = 0, Vip7 = 0, Vip8 = 0 },
        new ElligibleStakeModel { StakeUsdt = 150, StakeRmb = 550, Vip1 = 1, Vip2 = 2, Vip3 = 3, Vip4 = 5, Vip5 = 0, Vip6 = 0, Vip7 = 0, Vip8 = 0 },
        new ElligibleStakeModel { StakeUsdt = 180, StakeRmb = 600, Vip1 = 1, Vip2 = 2, Vip3 = 3, Vip4 = 5, Vip5 = 0, Vip6 = 0, Vip7 = 0, Vip8 = 0 },
        new ElligibleStakeModel { StakeUsdt = 200, StakeRmb = 650, Vip1 = 1, Vip2 = 2, Vip3 = 4, Vip4 = 5, Vip5 = 8, Vip6 = 10, Vip7 = 14, Vip8 = 15 },
    }
};

In [ ]:
Stake.ValidateStake()

In [ ]:
Sta

result

In [ ]:
List<bool> rowValueGeq = new List<bool> ();
for (int vipIndex = 0; vipIndex < results.Count - 1; vipIndex++)
{
    bool greaterThanOrEqual =  results[].Zip(results[vipIndex + 1], (prev, next) => next >= prev).All(x => x);
    rowValueGeq.Add(greaterThanOrEqual);

    if (!greaterThanOrEqual)
    {
        //Con
        Console.WriteLine($"Values for VIP{vipIndex + 2} are not greater than or equal to the previous row.");
        // You can take further actions if needed
    }
}

In [ ]:
List<List<int>> results = new List<List<int>>();
for (int i = 0; i < 8; i++)
{
    int vipIndex = i + 1 ;
    results.Add(Stake.Stake.Select(x => (int)x.GetType().GetProperty($"VIP{vipIndex}").GetValue(x)).ToList());
}
results

In [ ]:
CheckAscendingOrder(Stake.Stake)

In [ ]:
public ElligibleStakeModel2 GetVIOnly()
    {
        return new ElligibleStakeModel2 {
            VIP1 = VIP1,
            VIP2 = VIP2,
            VIP3 = VIP3,
            VIP4 = VIP4,
            VIP5 = VIP5,
            VIP6 = VIP6,
            VIP7 = VIP7,
            VIP8 = VIP8
        };
    }

In [ ]:
List<bool> rowValueGeq = new List<bool> ();
for (int vipIndex = 0; vipIndex < results.Count - 1; vipIndex++)
{
    bool greaterThanOrEqual =  results[vipIndex].Zip(results[vipIndex + 1], (prev, next) => next >= prev).All(x => x);
    rowValueGeq.Add(greaterThanOrEqual);

    if (!greaterThanOrEqual)
    {
        //Con
        Console.WriteLine($"Values for VIP{vipIndex + 2} are not greater than or equal to the previous row.");
        // You can take further actions if needed
    }
}

In [ ]:
rowValueGeq

In [ ]:
List<bool> rowValueGeq = new List<bool>();

for (int vipIndex = 0; vipIndex < results.Count - 1; vipIndex++)
{
    int lastNonZeroIndex = results[vipIndex].LastIndexOf(results[vipIndex].Where(x => x != 0).Last());

    bool greaterThanOrEqual = results[vipIndex].Take(lastNonZeroIndex + 1)
                             .Zip(results[vipIndex + 1].Take(lastNonZeroIndex + 1), (prev, next) => next >= prev)
                             .All(x => x);

    rowValueGeq.Add(greaterThanOrEqual);

    if (!greaterThanOrEqual)
    {
        Console.WriteLine("VIP Index " + vipIndex.ToString());
        Console.WriteLine($"Values for VIP{vipIndex + 2} are not greater than or equal to the previous row.");
        // You can take further actions if needed
    }
}


In [ ]:
results[3].LastIndexOf(results[3].Where(x => x != 0).Last())

In [ ]:
results[3]

In [ ]:
results[3].Take(3+ 1)
.Zip(results[3].Take(3+ 1), (prev, next) => next >= prev)
.All(x => x)


In [ ]:
results[3].Take(3 + 1)
.Zip(results[3 + 1].Take(3 + 1), (prev, next) => next >= prev)
.All(x => x)

In [ ]:
results[3 + 1].Take(3 + 1)

In [ ]:
List<bool> rowValueGeq = new List<bool>();

for (int vipIndex = 0; vipIndex < results.Count - 1; vipIndex++)
{
    // Find the last non-zero element index in the current row
    int lastNonZeroIndex = results[vipIndex].ToList().FindLastIndex(x => x != 0);

    // If there are no non-zero elements, consider the entire row
    if (lastNonZeroIndex == -1)
    {
        lastNonZeroIndex = results[vipIndex].Count - 1;
    }

    // Check if the values in the current and next rows are greater than or equal
    bool greaterThanOrEqual = results[vipIndex].Take(lastNonZeroIndex + 1)
                             .Zip(results[vipIndex + 1].Take(lastNonZeroIndex + 1), (prev, next) => next >= prev)
                             .All(x => x);

    rowValueGeq.Add(greaterThanOrEqual);

    if (!greaterThanOrEqual)
    {
        Console.WriteLine("VIP Index " + vipIndex.ToString());
        Console.WriteLine($"Values for VIP{vipIndex + 2} are not greater than or equal to the previous row.");
        // You can take further actions if needed
    }
}



In [ ]:
rowValueGeq

In [ ]:
List<bool> rowValueGeq = new List<bool>();

for (int vipIndex = 0; vipIndex < results.Count - 1; vipIndex++)
{
    // Find the index of the last non-zero element in the current row
    int lastNonZeroIndex = results[vipIndex].FindLastIndex(x => x != 0);

    // If there are no non-zero elements, consider the entire row
    if (lastNonZeroIndex == -1)
    {
        lastNonZeroIndex = results[vipIndex].Count - 1;
    }

    // Check if the values in the current and next rows are greater than or equal
    bool greaterThanOrEqual = results[vipIndex].Take(lastNonZeroIndex + 1)
                             .Zip(results[vipIndex + 1].Take(lastNonZeroIndex + 1), (prev, next) => next >= prev)
                             .All(x => x);

    rowValueGeq.Add(greaterThanOrEqual);

    if (!greaterThanOrEqual)
    {
        Console.WriteLine("VIP Index " + vipIndex.ToString());
        Console.WriteLine($"Values for VIP{vipIndex + 2} are not greater than or equal to the previous row.");
        // You can take further actions if needed
    }
}


In [ ]:
List<bool> rowValueGeq = new List<bool>();

for (int vipIndex = 0; vipIndex < results.Count - 1; vipIndex++)
{
    // Find the index of the last non-zero element in the current row
    int lastNonZeroIndex = results[vipIndex].FindLastIndex(x => x != 0);

    // If there are no non-zero elements, consider the entire row
    if (lastNonZeroIndex == -1)
    {
        lastNonZeroIndex = results[vipIndex].Count - 1;
    }

    // Check if the values in the current and next rows are greater than or equal
    bool greaterThanOrEqual = results[vipIndex].Take(lastNonZeroIndex + 1)
                             .Zip(results[vipIndex + 1].Take(lastNonZeroIndex + 1), (prev, next) => next >= 0 && next >= prev)
                             .All(x => x);

    rowValueGeq.Add(greaterThanOrEqual);

    if (!greaterThanOrEqual)
    {
        Console.WriteLine("VIP Index " + vipIndex.ToString());
        Console.WriteLine($"Values for VIP{vipIndex + 2} are not greater than or equal to the previous row.");
        // You can take further actions if needed
    }
}


In [ ]:
static bool CheckTable(int[][] table)
{
    // Iterate through rows
    for (int i = 1; i < table.Length; i++)
    {
        // Find the last non-zero element index in the current row
        int lastNonZeroIndex = Array.FindLastIndex(table[i], x => x != 0);

        // Compare elements up to the last non-zero element
        for (int j = 0; j <= lastNonZeroIndex; j++)
        {
            if (table[i][j] < table[i - 1][j])
            {
                // Current element is less than the previous row's element
                return false;
            }
        }
    }

    // All comparisons passed
    return true;
}


In [ ]:
int[][] table = new int[][]
{
    new int[] { 1, 1, 1, 1 },
    new int[] { 2, 2, 2, 2 },
    new int[] { 3, 3, 3, 4 },
    new int[] { 5, 5, 5, 5 },
    new int[] { 0, 0, 0, 8 },
    new int[] { 0, 0, 0, 11 },
    new int[] { 0, 0, 0, 12 },
    new int[] { 0, 0, 0, 15 }
};

// Perform the checks
bool result = CheckTable(table);

// Display the result
Console.WriteLine($"All rows satisfy the condition: {result}");

In [ ]:
result

In [ ]:
public class reqElligibleStakeMdl
    {
        public List<ElligibleStakeModel> Counts { get; set; } 

        public List<ElligibleStakeModelSeparated> SeparatedStakes {get; set;} = new List<ElligibleStakeModelSeparated>();

        public bool ValidateStake()
        {
            bool result = false;
            result = ValidateMaxFreeSpin() && CheckAscendingOrder() && CheckElligibleStakeValue();
            return result;
        }

        private bool ValidateMaxFreeSpin()
        {
            bool result = false;
            List<List<bool>> results = new List<List<bool>>();
            for (int i = 0; i < 8; i++)
            {
                int vipIndex = i + 1;
                results.Add(Counts.Select(x => x.GetType().GetProperty($"Vip{vipIndex}").GetValue(x) as int? > 15).ToList());
            }
            result = results.Any(x => x.Contains(true)) ? false : true;
            return result;
        }

        private bool CheckAscendingOrder()
        {
            int numRows = 8;
            int numCols = Counts.Count;

            for (int col = 0; col < numCols; col++)
            {
                int lastNonZeroIndex = -1;

                for (int row = 0; row < numRows; row++)
                {
                    if (Counts[col].GetVIPValue(row + 1) != 0)
                    {
                        lastNonZeroIndex = row + 1;
                    }
                }

                for (int row = 2; row <= lastNonZeroIndex; row++)
                {
                    if (Counts[col].GetVIPValue(row) < Counts[col].GetVIPValue(row - 1))
                    {
                        return false; 
                    }
                }
            }

            return true; 
        }

        private bool CheckElligibleStakeValue()
        {
            bool result = false;
            var usdtList = Counts.Select(x => x.StakeUsdt).ToList();
            var rmbList = Counts.Select(x => x.StakeRmb).ToList();
            result = rmbList.SequenceEqual(rmbList.OrderBy(x => x))
                        && rmbList.Distinct().Count() == rmbList.Count
                        && usdtList.SequenceEqual(usdtList.OrderBy(x => x))
                        && usdtList.Distinct().Count() == usdtList.Count;

            return result;
        }
        
        public void GetSeparatedStakeInfo(string userId){
            foreach (var stake in Counts){
                for (int i = 0; i<8;i++){
                    this.SeparatedStakes.Add(new ElligibleStakeModelSeparated { 
                        StakeAmtRmb = stake.StakeRmb,
                        StakeAmtUSDT = stake.StakeUsdt,
                        VIPLevel = i+1,
                        TotalSpin = stake.GetVIPValue(i+1),
                        User = userId
                    });
                }
            }

        }
    }

    public class ElligibleStakeModel
    {
        //todo add new draw model
        public int StakeUsdt { get; set; }
        public int StakeRmb { get; set; }
        public int Vip1 { get; set; }
        public int Vip2 { get; set; }
        public int Vip3 { get; set; }
        public int Vip4 { get; set; }
        public int Vip5 { get; set; }
        public int Vip6 { get; set; }
        public int Vip7 { get; set; }
        public int Vip8 { get; set; }

        public int GetVIPValue(int vipColumn)
        {
            switch (vipColumn)
            {
                case 1: return Vip1;
                case 2: return Vip2;
                case 3: return Vip3;
                case 4: return Vip4;
                case 5: return Vip5;
                case 6: return Vip6;
                case 7: return Vip7;
                case 8: return Vip8;
                default: throw new ArgumentOutOfRangeException(nameof(vipColumn), "Invalid VIP column number");
            }
        }

    }
    public class ElligibleStakeModelSeparated
    {
        //todo add new draw model
        public int StakeAmtUSDT { get; set; }
        public int StakeAmtRmb { get; set; }
        public int VIPLevel { get; set; }
        public int TotalSpin { get; set; }
        public string User {get; set;}

    }

In [ ]:
var Stake = new reqElligibleStakeMdl {
    Counts = new List<ElligibleStakeModel>
    {
        new ElligibleStakeModel { StakeUsdt = 149, StakeRmb = 500, Vip1 = 1, Vip2 = 2, Vip3 = 4, Vip4 = 5, Vip5 = 0, Vip6 = 0, Vip7 = 0, Vip8 = 0 },
        new ElligibleStakeModel { StakeUsdt = 150, StakeRmb = 550, Vip1 = 1, Vip2 = 2, Vip3 = 3, Vip4 = 5, Vip5 = 0, Vip6 = 0, Vip7 = 0, Vip8 = 0 },
        new ElligibleStakeModel { StakeUsdt = 180, StakeRmb = 600, Vip1 = 1, Vip2 = 2, Vip3 = 3, Vip4 = 5, Vip5 = 0, Vip6 = 0, Vip7 = 0, Vip8 = 0 },
        new ElligibleStakeModel { StakeUsdt = 200, StakeRmb = 650, Vip1 = 1, Vip2 = 2, Vip3 = 4, Vip4 = 5, Vip5 = 8, Vip6 = 10, Vip7 = 14, Vip8 = 15 },
    }
};

In [ ]:
Stake.ValidateStake()

In [ ]:
Stake.GetSeparatedStakeInfo("test");

In [ ]:
Stake.SeparatedStakes

In [ ]:
(int?)null

In [ ]:
var a = "abc";

In [ ]:
a.ToCharArray()[0]

In [ ]:
public class ElligibleStakeModel
{
    //todo add new draw model
    public decimal StakeUsdt { get; set; }
    public decimal StakeRmb { get; set; }
    public int Vip1 { get; set; }
    public int Vip2 { get; set; }
    public int Vip3 { get; set; }
    public int Vip4 { get; set; }
    public int Vip5 { get; set; }
    public int Vip6 { get; set; }
    public int Vip7 { get; set; }
    public int Vip8 { get; set; }

    public int GetVIPValue(int vipColumn)
    {
        switch (vipColumn)
        {
            case 1: return Vip1;
            case 2: return Vip2;
            case 3: return Vip3;
            case 4: return Vip4;
            case 5: return Vip5;
            case 6: return Vip6;
            case 7: return Vip7;
            case 8: return Vip8;
            default: throw new ArgumentOutOfRangeException(nameof(vipColumn), "Invalid VIP column number");
        }
    }
}


In [ ]:
public class SlotElligibleStakeSetting
{
    public decimal StakeAmtUSDT { get; set; }
    public decimal StakeAmtRmb { get; set; }
    public int VIPLevel { get; set; }
    public int TotalSpin { get; set; }
    public string UserId { get; set; }
    public long SpinSettingId { get; set; }
}

In [ ]:
public class reqElligibleStakeMdl
{
    public List<ElligibleStakeModel> Counts { get; set; } = new List<ElligibleStakeModel>();

    public List<SlotElligibleStakeSetting> SeparatedStakes { get; set; } = new List<SlotElligibleStakeSetting>();

    public bool ValidateStake()
    {
        bool result = false;
        result = ValidateMaxFreeSpin() && CheckAscendingOrder() && CheckElligibleStakeValue();
        return result;
    }

    private bool ValidateMaxFreeSpin()
    {
        bool result = false;
        List<List<bool>> results = new List<List<bool>>();
        for (int i = 0; i < 8; i++)
        {
            int vipIndex = i + 1;
            results.Add(Counts.Select(x => x.GetType().GetProperty($"Vip{vipIndex}").GetValue(x) as int? > 15).ToList());
        }
        result = results.Any(x => x.Contains(true)) ? false : true;
        return result;
    }

    private bool CheckAscendingOrder()
    {
        int numCols = Counts.Count;

        for (int col = 0; col < numCols; col++)
        {
            for (int row = 2; row <= 8; row++)
            {
                if (Counts[col].GetVIPValue(row) < Counts[col].GetVIPValue(row - 1))
                {
                    return false; 
                }
            }

            
        }

        for (int row = 1; row <= 8; row++)
        {
            for (int col = 1; col < numCols; col++)
            {
                if (Counts[col].GetVIPValue(row) < Counts[col-1].GetVIPValue(row))
                {
                    return false; 
                }
            }
        }

        return true; 
    }

    private bool CheckElligibleStakeValue()
    {
        bool result = false;
        var usdtList = Counts.Select(x => x.StakeUsdt).ToList();
        var rmbList = Counts.Select(x => x.StakeRmb).ToList();
        result = rmbList.SequenceEqual(rmbList.OrderBy(x => x))
                    && rmbList.Distinct().Count() == rmbList.Count
                    && usdtList.SequenceEqual(usdtList.OrderBy(x => x))
                    && usdtList.Distinct().Count() == usdtList.Count;

        return result;
    }

    public void GetSeparatedStakeInfo(string userId)
    {
        foreach (var stake in Counts)
        {
            for (int i = 0; i < 8; i++)
            {
                this.SeparatedStakes.Add(new SlotElligibleStakeSetting
                {
                    StakeAmtRmb = stake.StakeRmb,
                    StakeAmtUSDT = stake.StakeUsdt,
                    VIPLevel = i + 1,
                    TotalSpin = stake.GetVIPValue(i + 1),
                    UserId = userId
                });
            }
        }

    }
    public void SetSeparatedStakeSpinSettingId(long spinSettingId)
    {
        foreach (var stake in SeparatedStakes)
        {
            stake.SpinSettingId = spinSettingId;
        }

    }

    
}

In [ ]:
reqElligibleStakeMdl requestData = new reqElligibleStakeMdl
{
    Counts = new List<ElligibleStakeModel>
    {
        new ElligibleStakeModel { StakeUsdt = 100, StakeRmb = 500, Vip1 = 0, Vip2 = 1, Vip3 = 3, Vip4 = 4, Vip5 = 4, Vip6 = 5, Vip7 = 6, Vip8 = 7 },
        new ElligibleStakeModel { StakeUsdt = 150, StakeRmb = 600, Vip1 = 2, Vip2 = 2, Vip3 = 3, Vip4 = 4, Vip5 = 5, Vip6 = 6, Vip7 = 7, Vip8 = 0 }
    }
};
requestData.ValidateStake()

In [ ]:
Random randPrize = new Random();

In [ ]:
randPrize.Next(0, 1)

In [ ]:
using System;
using System.Globalization;

In [ ]:
static string TimeZoneConversion(string timeString, string userTimeZone, double dataTimeZone)
{
    // Extract sign, hours, and minutes
    bool isNegative = userTimeZone[0] == '-';
    int hours = int.Parse(userTimeZone.Substring(1, 1));
    int minutes = int.Parse(userTimeZone.Substring(3, 2));

    // Calculate total minutes
    int totalMinutes = (hours * 60 + minutes) * (isNegative ? -1 : 1);

    // Create TimeSpan
    // Parse user's time zone offset
    TimeSpan userTimeZoneOffset = TimeSpan.FromMinutes(totalMinutes);

    // Data time zone offset
    TimeSpan dbTimeZoneOffset = TimeSpan.FromHours(dataTimeZone);

    //Parse Data
    DateTime dbTime = DateTime.Parse(timeString);

    // Convert time to user's time zone
    DateTime userTime = dbTime + (userTimeZoneOffset - dbTimeZoneOffset);
    return userTime.ToString("dd/MM/yyyy HH:mm:ss");
}

In [ ]:
"yyyy-MM-dd HH:mm:ss"

In [ ]:
TimeZoneConversion(a, "+8:00", -4)

In [ ]:
string a = "13/03/2024 11:04:51";
string format = "yyyy-MM-dd HH:mm:ss";

In [ ]:
a = DateTime.ParseExact(a, format, CultureInfo.InvariantCulture).ToString()

In [ ]:
DateTime.Parse(a)

In [1]:
using System;
using System.Collections.Generic;
using System.Reflection;
// Define the Object class
public class MyObject {
    // Auto-implemented properties
    public int Id { get; set; }
    public string Name { get; set; }

    // Constructor
    public MyObject(int id, string name) {
        Id = id;
        Name = name;
    }

    // Method to display object information
    public void DisplayInfo() {
        Console.WriteLine($"Id: {Id}, Name: {Name}");
    }
    public Dictionary<int, string> ToDictionary() {
        Dictionary<int, string> dict = new Dictionary<int, string>();
        dict[Id] = Name;
        return dict;
    }
    public Dictionary<string, object> ToDictionary2() {
        Dictionary<string, object> dict = new Dictionary<string, object>();
        PropertyInfo[] properties = GetType().GetProperties(BindingFlags.Public | BindingFlags.Instance);
        foreach (PropertyInfo property in properties) {
            dict[property.Name] = property.GetValue(this);
        }
        return dict;
    }
}

In [2]:
MyObject obj = new MyObject(1, "Example");
obj.DisplayInfo();

Id: 1, Name: Example


In [ ]:
Dictionary<int, string> objDict = obj.ToDictionary();

In [ ]:
objDict 


In [ ]:
Dictionary<string, object> objDict = obj.ToDictionary2();

In [ ]:
var a = objDict.Values.ToList<object>();
a[0].GetType()

In [ ]:
List<MyObject> list = new List<MyObject>();

In [ ]:
MyObject obj2 = new MyObject(12, "Example2");
MyObject obj3 = new MyObject(13, "Example3");
MyObject obj4 = new MyObject(14, "Example4");
MyObject obj5 = new MyObject(15, "Example5");


In [ ]:
using System;

In [ ]:
int intValue = 42;
object boxedInt = intValue; // Boxing

In [ ]:
intValue

In [ ]:
boxedInt

In [ ]:
string sign;
nameof(sign)

In [ ]:
Dictionary<string, string> dict;


In [ ]:
using Newtonsoft.Json;

In [ ]:
string a = JsonConvert.SerializeObject(obj);
a

In [ ]:
JsonConvert.DeserializeObject<dynamic>(a)

In [ ]:
a.ToObject<Dictionary<string, object>>()

In [ ]:
dynamic jsonObj = DeserializeJson(a);

In [ ]:
string jsonString = "{\"name\": \"John\", \"age\": 30, \"isStudent\": true}";

dynamic jsonObject = System.Text.Json.JsonSerializer.Deserialize<dynamic>(jsonString);


In [ ]:
using System.Text.Json;

In [ ]:
jsonObject

In [ ]:
foreach (var property in jsonObject.EnumerateObject())
{
    Console.WriteLine($"{property.Name}: {GetPropertyStringValue(property.Value)}");
}

In [ ]:
static string GetPropertyStringValue(JsonElement element)
    {
        return element.ValueKind switch
        {
            JsonValueKind.String => element.GetString(),
            JsonValueKind.Number => element.GetDecimal().ToString(),
            JsonValueKind.True => "True",
            JsonValueKind.False => "False",
            _ => "Unknown Type"
        };
    }

In [ ]:
using System;
using System.Text.Json;

In [ ]:
string jsonString = "{\"name\": \"John\", \"age\": 30, \"isStudent\": true}";

dynamic jsonObject = JsonSerializer.Deserialize<dynamic>(jsonString);

// Accessing the "name" property
string name = jsonObject.name;

Console.WriteLine("Name: " + name);

In [ ]:
string jsonString = "{\"name\": \"John\", \"age\": 30, \"isStudent\": true}";

// Parse the JSON string into a JsonDocument
JsonDocument jsonObject = JsonDocument.Parse(jsonString);

// Accessing the "name" property
JsonElement root = jsonObject.RootElement;
string name = root.GetProperty("name").GetString();

Console.WriteLine("Name: " + name);

In [ ]:
int age = root.GetProperty("age").GetInt32();
Console.WriteLine("Age: " + age);

// Accessing the "isStudent" property and retrieving its value
bool isStudent = root.GetProperty("isStudent").GetBoolean();
Console.WriteLine("Is Student: " + isStudent);

In [ ]:
using Newtonsoft.Json;
using System;


In [ ]:
static dynamic DeserializeDynamicObject(string json)
{
    return JsonConvert.DeserializeObject<dynamic>(json);
}

In [ ]:
string unknownFormatString = "{\"name\": \"John\", \"age\": 30, \"city\": \"New York\"}";

dynamic dynamicObject = DeserializeDynamicObject(unknownFormatString);

Console.WriteLine($"Name: {dynamicObject.name}");
Console.WriteLine($"Age: {dynamicObject.age}");
Console.WriteLine($"City: {dynamicObject.city}");

In [ ]:
dynamicObject[name]

In [ ]:
dynamicObject.city.ToString()

In [ ]:
string unknownFormatString = "{\"name\": \"John\", \"age\": 30, \"city\": \"New York\"}";

dynamic dynamicObject = DeserializeDynamicObject(unknownFormatString);

Console.WriteLine($"Name: {dynamicObject["name"]}");
Console.WriteLine($"Age: {dynamicObject["age"]}");
Console.WriteLine($"City: {dynamicObject["city"]}");

In [ ]:
dynamicObject["name"].ToString()

In [ ]:
var (_, _, _, pop1, _, pop2) = QueryCityDataForYears("New York City", 1960, 2010);

Console.WriteLine($"Population change, 1960 to 2010: {pop2 - pop1:N0}");

static (string, double, int, int, int, int) QueryCityDataForYears(string name, int year1, int year2)
{
    int population1 = 0, population2 = 0;
    double area = 0;

    if (name == "New York City")
    {
        area = 468.48;
        if (year1 == 1960)
        {
            population1 = 7781984;
        }
        if (year2 == 2010)
        {
            population2 = 8175133;
        }
        return (name, area, year1, population1, year2, population2);
    }

    return ("", 0, 0, 0, 0, 0);
}
// The example displays the following output:
//      Population change, 1960 to 2010: 393,149

In [ ]:
string message = "kEyNaME";
char.ToUpper(message.ToLower()[0]) + message.ToLower().Substring(1)

In [ ]:
Dictionary<string, int> myDictionary = new Dictionary<string, int>();

// Populate the dictionary with some data
myDictionary.Add("apple", 10);
myDictionary.Add("cherrys", 14);
myDictionary.Add("banana", 20);
myDictionary.Add("cherry", 5);
myDictionary.Add("orange", 15);

// Order the dictionary by key name in ascending order
var orderedByKeyAscending = myDictionary.OrderBy(pair => pair.Key);

Console.WriteLine("Dictionary ordered by key name in ascending order:");
foreach (var pair in orderedByKeyAscending)
{
    Console.WriteLine($"Key: {pair.Key}, Value: {pair.Value}");
}

// Order the dictionary by key name in descending order
var orderedByKeyDescending = myDictionary.OrderByDescending(pair => pair.Key);

Console.WriteLine("\nDictionary ordered by key name in descending order:");
foreach (var pair in orderedByKeyDescending)
{
    Console.WriteLine($"Key: {pair.Key}, Value: {pair.Value}");
}

In [ ]:
orderedByKeyAscending

In [ ]:
myDictionary 

In [ ]:
myDictionary.OrderBy(pair => pair.Value)

In [ ]:
var a = myDictionary.OrderBy(pair => pair.Value).ToDictionary(kvp => kvp.Key, kvp => kvp.Value);

In [ ]:
a["cherry"]

In [ ]:
a

In [3]:
obj

Id,1
Name,Example


In [40]:
public Dictionary<string, T> _GenerateDictionary<T>(
            MyObject _obj)
{
    Console.WriteLine(typeof(T).Name);
    Dictionary<string, T> dict = new Dictionary<string, T>();
    dict.Add(_obj.Name, (T)Convert.ChangeType(obj.Id, typeof(T)));
    
    return dict;


    
}

In [41]:
typeof(a["Example"])

Error: (1,8): error CS0246: The type or namespace name 'a' could not be found (are you missing a using directive or an assembly reference?)
(1,9): error CS0270: Array size cannot be specified in a variable declaration (try initializing with a 'new' expression)

In [42]:
(object)obj.Id.ToString()

1

In [43]:
var a = _GenerateDictionary<string>(obj);

String


In [44]:
var b= a["Example"];

In [45]:
b.GetType()

System.String

In [1]:
public static Type GetTypeFromInput(string input)
        {
            Dictionary<string, Type> typeMappings = new Dictionary<string, Type>
            {
                { "int", typeof(int) },
                { "string", typeof(string) },
                { "double", typeof(double) },
                { "long", typeof(long) },
                { "float", typeof(float) },
                { "decimal", typeof(decimal) },
                { "bool", typeof(bool) },
                { "char", typeof(char) },
                { "byte", typeof(byte) },
                { "Array", typeof(Array) },
                { "DateTime", typeof(DateTime) },
                { "TimeSpan", typeof(TimeSpan) },
                { "object", typeof(object) }
            };

            if (typeMappings.ContainsKey(input))
            {
                return typeMappings[input];
            }
            else
            {
                throw new ArgumentException($"Type mapping not found for input: {input}");
            }
        }

In [33]:
var a = "1";
var c = GetTypeFromInput("decimal");



In [13]:
public static object ConvertValue(string value, Type targetType)
{
    return Convert.ChangeType(value, targetType);
}

In [46]:
var b = ConvertValue(a, c);
var e = GetTypeFromInput("decimal");

var d = ConvertValue(b.ToString(), e);

In [67]:
d.GetType()

System.Decimal

In [60]:
Console.WriteLine(d)

0.00


In [58]:
d.ToString("F")

0.000

In [57]:
decimal d = 0.00m;

In [63]:
var x = "1.01";

In [64]:
var e = GetTypeFromInput("decimal");

In [65]:
var d = ConvertValue(x, e);

In [68]:
d.ToString("F")

Error: (1,3): error CS1501: No overload for method 'ToString' takes 1 arguments